### **Synthetic Data Generation**

In [ ]:
!pip install Faker==18.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.8 MB/s eta 0:00:00


### UK_cities table

In [ ]:
#to delete/reinitialize table
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM UK_cities")
conn.commit()

In [ ]:
import sqlite3
from faker import Faker
import random

# Connect to SQLite database
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

# Create UK_cities table
cursor.execute('''
CREATE TABLE IF NOT EXISTS UK_cities (
  cities_id INTEGER PRIMARY KEY AUTOINCREMENT,
  cities_name TEXT
);
''')

# Sample list of UK cities
cities_name = [
    "London", "Birmingham", "Manchester", "Liverpool", "Leeds",
    "Sheffield", "Bristol", "Newcastle upon Tyne", "Nottingham", "Leicester",
    "Coventry", "Bradford", "Southampton", "Derby", "Stoke-on-Trent",
    "Wolverhampton", "Plymouth", "Reading", "Kingston upon Hull", "Belfast",
    "Edinburgh", "Glasgow", "Cardiff", "Swansea", "Aberdeen",
    "Dundee", "Portsmouth", "York", "Exeter", "Cambridge"
]

# Initialize cities_id counter
cities_id_counter = 1

for city in cities_name:
  try:
    cursor.execute("INSERT INTO UK_cities (cities_id, cities_name) VALUES (?, ?)", (cities_id_counter, city))
    cities_id_counter += 1
  except sqlite3.IntegrityError:
    print(f"City '{city}' already exists in the database.")

conn.commit()
conn.close()
print("UK_cities table populated with fake data.")

UK_cities table populated with fake data.


In [ ]:
import sqlite3
import random

# Print all data from the table
print("UK Cities table data:")

conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM UK_cities")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

UK Cities table data:
(1, 'London')
(2, 'Birmingham')
(3, 'Manchester')
(4, 'Liverpool')
(5, 'Leeds')
(6, 'Sheffield')
(7, 'Bristol')
(8, 'Newcastle upon Tyne')
(9, 'Nottingham')
(10, 'Leicester')
(11, 'Coventry')
(12, 'Bradford')
(13, 'Southampton')
(14, 'Derby')
(15, 'Stoke-on-Trent')
(16, 'Wolverhampton')
(17, 'Plymouth')
(18, 'Reading')
(19, 'Kingston upon Hull')
(20, 'Belfast')
(21, 'Edinburgh')
(22, 'Glasgow')
(23, 'Cardiff')
(24, 'Swansea')
(25, 'Aberdeen')
(26, 'Dundee')
(27, 'Portsmouth')
(28, 'York')
(29, 'Exeter')
(30, 'Cambridge')


### Users table

In [ ]:
#to delete/reinitialize table
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM Users")
conn.commit()

In [ ]:
import sqlite3
import random
from faker import Faker

# Initialize Faker
fake = Faker()

# Connect to the database
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

# Create Users table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS Users (
  user_id INTEGER PRIMARY KEY,
  email TEXT,
  first_name TEXT,
  last_name TEXT,
  phone_num TEXT,
  registration_date TEXT,
  gender TEXT CHECK (gender IN ('Male', 'Female')),
  user_type TEXT CHECK (user_type IN ('Guest', 'Host')),
  cities_id INTEGER NOT NULL,
  FOREIGN KEY (cities_id) REFERENCES UK_cities (cities_id)
);
''')
#gender - 0: Male, 1: Female
#user_type - 0: Guest, 1: Host

# Function to generate a random phone number
def generate_phone_number():
    first_digit = 0
    remaining_digits = [random.randint(0, 9) for _ in range(9)]
    return str(first_digit) + ''.join(map(str, remaining_digits))

from datetime import date, timedelta
   # Get today's date
today = date.today()
   # Calculate the date one year ago
one_year_ago = today - timedelta(days=365)
   # Generate a date within the last year
date_within_last_year = fake.date_between(start_date=one_year_ago, end_date=today)

# Generate users
users_data = []
# Initialize user_id counter
user_id_counter = 1
cursor.execute("SELECT cities_id FROM UK_cities")
available_cities_ids = [row[0] for row in cursor.fetchall()]

for _ in range(500):
    gender_choice = random.choice(["M", "F"])
    first_name = fake.first_name_male() if gender_choice =="M" else fake.first_name_female()
    last_name = fake.last_name()
    email = f"{first_name[0].lower()}.{last_name.lower()}@{fake.domain_name()}"
    phone_num = generate_phone_number()
    registration_date = fake.date_between(start_date=one_year_ago, end_date=today).isoformat()
    gender = 'Male' if gender_choice == "M" else 'Female'
    user_type = 'Guest' if random.choice([0, 1]) == 0 else 'Host'
    cities_id = random.choice(available_cities_ids)
    users_data.append((user_id_counter, email, first_name, last_name, phone_num, registration_date, gender, user_type, cities_id))
    user_id_counter += 1

# Insert user data with manually managed user_id and error handling
try:
  cursor.executemany("""
            INSERT INTO Users (user_id, email, first_name, last_name, phone_num, registration_date, gender, user_type, cities_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, users_data)
  conn.commit()
#except sqlite3.IntegrityError:
 #  print(f"Error inserting user: {user_data}. user_id might already exist.")
#except Exception as e:
 #  print(f"An unexpected error occurred: {e}")
except sqlite3.IntegrityError as e:
           if "UNIQUE constraint failed" in str(e):  # Check for the specific error message
               print(f"Error inserting user: {users_data}. user_id might already exist.")
           else:
               raise  # Re-raise other exceptions
# Close the connection
conn.close()

print("Users table populated with fake data.")

Users table populated with fake data.


In [ ]:
import sqlite3
import random

# Print all data from the table
print("Users table data:")

conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM Users")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

Users table data:
(1, 'a.kennedy@cole.com', 'Anita', 'Kennedy', '0826869774', '2024-09-17', 'Female', 'Host', 24)
(2, 'l.martinez@love.com', 'Linda', 'Martinez', '0179147742', '2024-04-18', 'Female', 'Guest', 27)
(3, 'a.williams@kim.com', 'Angela', 'Williams', '0772297048', '2025-01-23', 'Female', 'Guest', 7)
(4, 'c.moore@perez.com', 'Christine', 'Moore', '0905975500', '2024-10-03', 'Female', 'Guest', 22)
(5, 'w.thompson@moses.com', 'William', 'Thompson', '0841025055', '2025-02-27', 'Male', 'Guest', 19)
(6, 'a.clark@hart.com', 'Angela', 'Clark', '0155300347', '2024-07-10', 'Female', 'Host', 2)
(7, 'b.mitchell@miller-pacheco.com', 'Bruce', 'Mitchell', '0466697132', '2024-12-29', 'Male', 'Host', 6)
(8, 'g.smith@vasquez-figueroa.com', 'Gabrielle', 'Smith', '0771180379', '2024-09-11', 'Female', 'Guest', 2)
(9, 'k.robles@white.com', 'Kerri', 'Robles', '0008100152', '2024-06-26', 'Female', 'Host', 28)
(10, 'c.pope@williams-smith.com', 'Carolyn', 'Pope', '0632601538', '2024-08-28', 'Female', 

### Listings table

In [ ]:
#to delete/reinitialize table
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM Listings")
conn.commit()

In [ ]:
import sqlite3
import random
from faker import Faker

# Initialize Faker
fake = Faker()

# Connect to the database
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

# Create Listings table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS Listings (
  listing_id INTEGER PRIMARY KEY,
  title TEXT,
  description TEXT,
  property_type TEXT,
  street TEXT,
  house_num TEXT,
  price_per_night DECIMAL,
  availability_status TEXT CHECK (availability_status IN ('available', 'booked')),
  num_bedrooms INTEGER,
  num_bathrooms INTEGER,
  max_guests INTEGER,
  min_nights INTEGER,
  listing_date TEXT,
  num_pics INTEGER,
  host_id INTEGER NOT NULL,
  cities_id INTEGER NOT NULL,
  FOREIGN KEY (host_id) REFERENCES Users (user_id),
  FOREIGN KEY (cities_id) REFERENCES UK_cities (cities_id)
);
''')

# Initialize listing_id counter
listing_id_counter = 1

# Generate listings data
listings_data = []
cursor.execute("SELECT user_id FROM Users WHERE user_type = 'Host'")  # Get host IDs
available_host_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT cities_id FROM UK_cities")
available_cities_ids = [row[0] for row in cursor.fetchall()]

property_types = ["Apartment", "House", "Studio", "Cottage", "Villa"]

property_keywords = {
    "Apartment": ["cozy", "modern", "city view", "studio", "balcony", "spacious", "well-equipped", "central location", "quiet neighborhood"],
    "House": ["spacious", "family-friendly", "garden", "fireplace", "garage", "private", "comfortable", "large kitchen", "multiple bedrooms"],
    "Studio": ["compact", "efficient", "minimalist", "city center", "affordable", "stylish", "open-plan", "great for students", "close to amenities"],
    "Cottage": ["charming", "rustic", "cozy", "garden", "countryside", "traditional", "fireplace", "peaceful", "scenic views"],
    "Villa": ["luxury", "spacious", "private pool", "garden", "sea view", "modern", "fully equipped", "exclusive", "breathtaking views"]
}

property_bedrooms = {
    "Apartment": (1, 3),
    "House": (2, 5),
    "Studio": (1),
    "Cottage": (1, 3),
    "Villa": (3, 6)
}

from datetime import date, timedelta
   # Get today's date
today = date.today()
   # Calculate the date one year ago
one_year_ago = today - timedelta(days=365)
   # Generate a date within the last year
date_within_last_year = fake.date_between(start_date=one_year_ago, end_date=today)

def get_max_guests(num_bedrooms):
    if num_bedrooms in range(1, 6):
        return {1: 2, 2: 4, 3: 6, 4: 8, 5: 10}.get(num_bedrooms)
    else:
        return 15  # Default max_guests if num_bedrooms is outside the range

def get_num_bathrooms(num_bedrooms):
    if num_bedrooms in range(1, 6):
        return {1: 1, 2: 1, 3: 2, 4: 3, 5: 4}.get(num_bedrooms)
    else:
        return 5

for _ in range(500):
    property_type = random.choice(property_types)
    keywords = property_keywords.get(property_type, [])  # Get keywords for property type
    title = f"{random.choice(keywords)} {property_type}"
    description = f"This {property_type} is {random.choice(keywords)} and features a {random.choice(keywords)}. It is located in a {random.choice(keywords)} area and is perfect for {random.choice(['families', 'couples', 'solo travelers'])}."
    street = fake.street_name()
    house_num = str(random.randint(1, 100))
    availability_status = random.choice(['available', 'booked'])

    bedroom_range = property_bedrooms.get(property_type)
    if isinstance(bedroom_range, tuple):
        if len(bedroom_range) == 2:  # For ranges (min, max)
            num_bedrooms = random.randint(bedroom_range[0], bedroom_range[1])
        elif len(bedroom_range) == 1:  # For fixed values (single tuple element)
            num_bedrooms = bedroom_range[0]
    else:  # If property_type not in property_bedrooms, use a default
        num_bedrooms = 2
    max_guests = get_max_guests(num_bedrooms)
    num_bathrooms = get_num_bathrooms(num_bedrooms)
    min_nights = random.randint(1, 3)

    base_price = 50  # Starting price
    price_per_night = base_price + \
                  int(min_nights) * 5 + \
                  int(max_guests) * 10 + \
                  int(num_bedrooms) * 20
    price_per_night = round(price_per_night, 2)

    listing_date = fake.date_between(start_date=one_year_ago, end_date=today)
    num_pics = random.randint(1, 10)
    host_id = random.choice(available_host_ids)
    cities_id = random.choice(available_cities_ids)

    listings_data.append((listing_id_counter,title, description, property_type, street, house_num, price_per_night,
                         availability_status, num_bedrooms, num_bathrooms,
                         max_guests, min_nights, listing_date, num_pics, host_id, cities_id))
    listing_id_counter += 1
# Insert listings data
try:
    cursor.executemany("""
        INSERT INTO Listings (listing_id, title, description, property_type, street, house_num, price_per_night,
                            availability_status, num_bedrooms, num_bathrooms,
                            max_guests, min_nights, listing_date, num_pics, host_id, cities_id)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, listings_data)
    listing_id_counter += 1
    conn.commit()
except sqlite3.IntegrityError:
      print(f"Error inserting user: {listings_data}. listing_id might already exist.")
except Exception as e:
      print(f"An unexpected error occurred: {e}")

# Close the connection
conn.close()

print("Listings table populated with fake data.")

Listings table populated with fake data.


In [ ]:
import sqlite3
import random

# Print all data from the table
print("Listings table data:")

conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM Listings")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

Listings table data:
(1, 'exclusive Villa', 'This Villa is modern and features a garden. It is located in a garden area and is perfect for solo travelers.', 'Villa', 'Elizabeth Common', '17', 225, 'booked', 4, 3, 8, 3, '2024-06-18', 9, 66, 29)
(2, 'minimalist Studio', 'This Studio is affordable and features a efficient. It is located in a close to amenities area and is perfect for families.', 'Studio', 'Watson Gateway', '71', 140, 'available', 2, 1, 4, 2, '2024-10-21', 5, 429, 15)
(3, 'scenic views Cottage', 'This Cottage is scenic views and features a charming. It is located in a peaceful area and is perfect for solo travelers.', 'Cottage', 'Baker Ville', '54', 145, 'booked', 2, 1, 4, 3, '2024-09-02', 6, 147, 25)
(4, 'modern Apartment', 'This Apartment is cozy and features a city view. It is located in a central location area and is perfect for families.', 'Apartment', 'Alexis Flat', '96', 175, 'available', 3, 2, 6, 1, '2024-04-07', 9, 139, 13)
(5, 'modern Apartment', 'This Apartment 

### Bookings table

In [ ]:
#to delete/reinitialize table
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM Bookings")
conn.commit()

In [ ]:
import sqlite3
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Connect to the database
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

# Create Bookings table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Bookings (
  booking_id INTEGER PRIMARY KEY,
  check_in_date TEXT,
  check_out_date TEXT,
  booking_status TEXT CHECK (booking_status IN ('confirmed', 'cancelled by host', 'cancelled by guest', 'pending')),
  booking_date TEXT,
  host_response_date TEXT,
  host_response_status TEXT CHECK (host_response_status IN ('accepted', 'rejected', 'pending')),
  num_guests INTEGER,
  guest_id INTEGER NOT NULL,
  listing_id INTEGER NOT NULL,
  FOREIGN KEY (guest_id) REFERENCES Users (user_id),
  FOREIGN KEY (listing_id) REFERENCES Listings (listing_id)
);
''')

# Get available guest and listing IDs
cursor.execute("SELECT user_id FROM Users WHERE user_type = 'Guest'")
available_guest_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT listing_id FROM Listings")
available_listing_ids = [row[0] for row in cursor.fetchall()]

# Generate bookings data
bookings_data = []
booking_id_counter = 1
booked_listings = set()  # Keep track of booked listings to avoid double bookings

today = date.today()
one_year_ago = today - timedelta(days=365)

for _ in range(500):
    listing_id = random.choice(available_listing_ids)

    # Fetch listing_date for the selected listing_id
    cursor.execute("SELECT listing_date FROM Listings WHERE listing_id = ?", (listing_id,))
    listing_date_str = cursor.fetchone()[0]
    listing_date = datetime.fromisoformat(listing_date_str).date()

    # Generate random dates within a reasonable range
    check_in_date = fake.date_between(start_date=one_year_ago, end_date=today)
    booking_date = fake.date_between(start_date=one_year_ago, end_date=min(check_in_date, today))
    check_out_date = check_in_date + timedelta(days=random.randint(1, 21))

    cursor.execute("SELECT availability_status FROM Listings WHERE listing_id = ?", (listing_id,))
    availability_status = cursor.fetchone()[0]

    if availability_status == 'booked':
      booking_status = random.choice(['confirmed'])
    elif availability_status == 'available':
      booking_status = random.choice(['cancelled by host', 'cancelled by guest','pending'])

    # Map booking_status to host_response_status and host_response_date
    response_logic = {
    'confirmed': ('accepted', booking_date + timedelta(days=random.randint(1, 7))),
    'pending': ('pending', None),
    'cancelled by host': (random.choice(['rejected', 'pending']), booking_date + timedelta(days=random.randint(1, 7))),  # Always calculate host_response_date for 'cancelled by host'
    'cancelled by guest': (random.choice(['rejected', 'pending']), None),
    }

    # Get host_response_status and host_response_date
    host_response_status, host_response_date = response_logic.get(booking_status, ('pending', None))

    if booking_status == 'cancelled by host' and host_response_status == 'pending':
      host_response_date = None  # Set to None if cancelled by host and pending

    # Handle 'cancelled by guest' (add timedelta if response not pending)
    if booking_status == 'cancelled by guest' and host_response_status != 'pending':
      host_response_date = booking_date + timedelta(days=random.randint(1, 7))

    # Prevent same guest from double-booking:
    while (guest_id, check_in_date, check_out_date) in booked_listings:
        guest_id = random.choice(available_guest_ids)  # Choose a different guest
        booked_listings.add((guest_id, check_in_date, check_out_date))

    # 2. Prevent different guests from overlapping bookings:
    while any(
        (listing_id, existing_check_in, existing_check_out) in booked_listings
        for existing_check_in in [check_in_date + timedelta(days=i) for i in range((check_out_date - check_in_date).days +1)]  # Enumerate booking range
         for existing_check_out in [check_in_date + timedelta(days=i) for i in range((check_out_date - check_in_date).days +1)] # Enumerate booking range
         if existing_check_in <= existing_check_out # Ensure valid date range
    ):
        check_in_date = fake.date_between(start_date=one_year_ago, end_date=today)
        check_out_date = check_in_date + timedelta(days=random.randint(1, 21))

    # Record the booking to prevent future conflicts:
    for booking_day in range((check_out_date - check_in_date).days + 1):
        booked_listings.add((listing_id, check_in_date + timedelta(days=booking_day), check_out_date))


    # Generate num_guests considering only max_guests
    cursor.execute("SELECT max_guests FROM Listings WHERE listing_id = ?", (listing_id,))
    max_guests = cursor.fetchone()[0]
    num_guests = random.randint(1, max_guests)

    bookings_data.append((booking_id_counter, check_in_date.isoformat(), check_out_date.isoformat(), booking_status,
                          booking_date.isoformat(), host_response_date.isoformat() if host_response_date else None,
                          host_response_status, num_guests, guest_id, listing_id))
    booking_id_counter += 1

# Insert bookings data
try:
    cursor.executemany("""
        INSERT INTO Bookings (booking_id, check_in_date, check_out_date, booking_status, booking_date,
                              host_response_date, host_response_status, num_guests, guest_id, listing_id)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, bookings_data)
    conn.commit()
except sqlite3.IntegrityError as e:
    if "UNIQUE constraint failed" in str(e):  # Check for duplicate booking IDs
        print(f"Error inserting data: booking_id might already exist.")
    else:
        raise  # Re-raise other exceptions
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Close the connection
conn.close()
print("Bookings table populated with fake data.")


Bookings table populated with fake data.


In [ ]:
import sqlite3
import random

# Print all data from the table
print("Bookings table data:")

conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM Bookings")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

Bookings table data:
(1, '2025-01-04', '2025-01-20', 'pending', '2024-05-11', None, 'pending', 10, 478, 208)
(2, '2024-06-11', '2024-06-29', 'confirmed', '2024-04-02', '2024-04-04', 'accepted', 4, 478, 240)
(3, '2025-03-02', '2025-03-05', 'confirmed', '2024-07-25', '2024-07-31', 'accepted', 3, 478, 288)
(4, '2024-12-03', '2024-12-20', 'cancelled by host', '2024-05-11', '2024-05-17', 'rejected', 1, 478, 498)
(5, '2024-07-24', '2024-07-27', 'cancelled by guest', '2024-05-08', '2024-05-11', 'rejected', 3, 478, 227)
(6, '2024-06-09', '2024-06-27', 'confirmed', '2024-05-20', '2024-05-23', 'accepted', 3, 478, 292)
(7, '2024-06-09', '2024-06-13', 'confirmed', '2024-04-11', '2024-04-16', 'accepted', 1, 478, 176)
(8, '2024-06-23', '2024-07-08', 'cancelled by host', '2024-04-17', None, 'pending', 2, 478, 230)
(9, '2024-04-26', '2024-05-02', 'confirmed', '2024-03-21', '2024-03-28', 'accepted', 5, 478, 285)
(10, '2025-03-16', '2025-04-02', 'confirmed', '2024-04-11', '2024-04-14', 'accepted', 3, 47

### Payment table

In [ ]:
#to delete/reinitialize table
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM Payments")
conn.commit()

In [ ]:
import sqlite3
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Connect to the database
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

# Create Payments table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Payments (
  payment_id INTEGER PRIMARY KEY,
  payment_method TEXT,
  payment_status TEXT CHECK (payment_status IN ('paid', 'pending', 'failed')),
  payment_amount DECIMAL,
  payment_date TEXT,
  booking_id INTEGER NOT NULL,
  FOREIGN KEY (booking_id) REFERENCES Bookings (booking_id)
);
''')
#payment_status - 0: paid, 1: pending, 2: failed

# Get available booking IDs
cursor.execute("SELECT booking_id FROM Bookings")
available_booking_ids = [row[0] for row in cursor.fetchall()]

# Generate payments data
payments_data = []
payment_id_counter = 1
for booking_id in available_booking_ids:
    payment_method = random.choice(["Credit/Debit Card", "PayPal", "Bank Transfer"])
    payment_status = random.choice(['paid', 'pending', 'failed'])

    # Fetch price_per_night and num_guests from Listings and Bookings tables
    cursor.execute("""
        SELECT L.price_per_night, B.num_guests
        FROM Listings L
        JOIN Bookings B ON L.listing_id = B.listing_id
        WHERE B.booking_id = ?
    """, (booking_id,))
    price_per_night, num_guests = cursor.fetchone()

    # Calculate payment amount
    payment_amount = price_per_night * num_guests

    # Generate payment date (around booking date)
    cursor.execute("SELECT booking_date FROM Bookings WHERE booking_id = ?", (booking_id,))
    booking_date_str = cursor.fetchone()[0]
    booking_date = datetime.fromisoformat(booking_date_str).date()
    payment_date = fake.date_between(start_date=booking_date - timedelta(days=3), end_date=booking_date + timedelta(days=3))

    # Set payment_status based on booking_status
    cursor.execute("SELECT booking_status FROM Bookings WHERE booking_id = ?", (booking_id,))
    booking_status = cursor.fetchone()[0]
    if booking_status == 'confirmed':
        payment_status = 'paid'
    elif booking_status == 'pending':
        payment_status = 'pending'
    else:
        payment_status = random.choice(['failed', 'pending'])

    payment_id_counter += 1
    payments_data.append((payment_id_counter, payment_method, payment_status, payment_amount, payment_date.isoformat(), booking_id))

# Insert payments data using executemany
try:
  cursor.executemany("""
    INSERT INTO Payments (payment_id, payment_method, payment_status, payment_amount, payment_date, booking_id)
    VALUES (?,?, ?, ?, ?, ?)
""", payments_data)
  conn.commit()
except sqlite3.IntegrityError:
  print(f"Error inserting user: {payments_data}. payment_id might already exist.")
except Exception as e:
  print(f"An unexpected error occurred: {e}")
conn.close()

print("Payments table populated with fake data.")

Payments table populated with fake data.


In [ ]:
import sqlite3
import random

# Print all data from the table
print("Payments table data:")

conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM Payments")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

Payments table data:
(2, 'PayPal', 'pending', 2550, '2024-05-11', 1)
(3, 'Credit/Debit Card', 'paid', 880, '2024-04-04', 2)
(4, 'Bank Transfer', 'paid', 555, '2024-07-25', 3)
(5, 'Bank Transfer', 'failed', 100, '2024-05-09', 4)
(6, 'PayPal', 'failed', 420, '2024-05-09', 5)
(7, 'PayPal', 'paid', 405, '2024-05-19', 6)
(8, 'Credit/Debit Card', 'paid', 105, '2024-04-10', 7)
(9, 'PayPal', 'failed', 270, '2024-04-19', 8)
(10, 'PayPal', 'paid', 1075, '2024-03-20', 9)
(11, 'PayPal', 'paid', 405, '2024-04-10', 10)
(12, 'Credit/Debit Card', 'failed', 140, '2024-04-05', 11)
(13, 'Bank Transfer', 'paid', 140, '2024-04-12', 12)
(14, 'Bank Transfer', 'pending', 215, '2025-01-31', 13)
(15, 'Credit/Debit Card', 'paid', 135, '2024-03-28', 14)
(16, 'PayPal', 'paid', 3630, '2024-04-03', 15)
(17, 'Bank Transfer', 'pending', 2680, '2024-03-18', 16)
(18, 'PayPal', 'paid', 580, '2024-10-16', 17)
(19, 'Bank Transfer', 'pending', 405, '2024-05-10', 18)
(20, 'Credit/Debit Card', 'paid', 225, '2024-05-31', 19)
(

### Reviews table

In [ ]:
#to delete/reinitialize table
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM Reviews")
conn.commit()

In [ ]:
import sqlite3
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Connect to the database
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

# Create Reviews table (if not exists)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Reviews (
  review_id INTEGER PRIMARY KEY,
  rating_score INTEGER,
  review_text TEXT,
  review_date TEXT,
  response_text TEXT,
  response_date TEXT,
  booking_id INTEGER NOT NULL,
  listing_id INTEGER NOT NULL,
  FOREIGN KEY (booking_id) REFERENCES Bookings (booking_id),
  FOREIGN KEY (listing_id) REFERENCES Listings (listing_id)
);
''')

# Get available booking and listing IDs
cursor.execute("SELECT DISTINCT listing_id FROM Bookings")  # Get both booking_id and listing_id
available_bookings_listings = [(row[0]) for row in cursor.fetchall()]

# Generate reviews data
reviews_data = []
review_id_counter = 1

for listing_id in available_bookings_listings:
    num_reviews = random.randint(1,3)
    for _ in range(num_reviews):
      cursor.execute("SELECT booking_id FROM Bookings WHERE listing_id = ?", (listing_id,))
      booking_ids_for_listing = [row[0] for row in cursor.fetchall()]
      booking_id = random.choice(booking_ids_for_listing)
      #rating_score = random.randint(1, 5)
      # Define weights for each rating score
      weights = [1, 2, 3, 4, 5]  # Increase the weight for higher ratings
# rating_score = random.randint(1, 5)  # Original line
      rating_score = random.choices(range(1, 6), weights=weights)[0]
      if rating_score <= 2:  # Consider ratings 1 or 2 as negative
            sentiment = "negative"
      else:
            sentiment = "positive"
      review_text = fake.paragraph(nb_sentences=3)

      # Get check_out_date for the booking
      cursor.execute("SELECT check_out_date FROM Bookings WHERE booking_id = ?", (booking_id,))
      check_out_date_str = cursor.fetchone()[0]
      check_out_date = datetime.fromisoformat(check_out_date_str).date()

      # Generate review_date after check_out_date
      review_date = fake.date_between(start_date=check_out_date + timedelta(days=1), end_date=check_out_date + timedelta(days=31))

    # Generate review text based on sentiment and listing description
    #  cursor.execute("SELECT description FROM Listings WHERE listing_id = ?", (listing_id,))
    #  listing_description = cursor.fetchone()[0]
    #  if sentiment == "positive":
    #    review_text = f"I had a wonderful stay! The {random.choice(listing_description.split())} was fantastic, and the overall experience was great. I would definitely recommend this listing."
    #  else:
    #    review_text = f"Unfortunately, my stay was not as expected. The {random.choice(listing_description.split())} was disappointing, and I encountered some issues during my stay. I would not recommend this listing."

      # Get listing details for review generation
      cursor.execute("SELECT property_type FROM Listings WHERE listing_id = ?", (listing_id,))
      property_type = cursor.fetchone()[0]

      # Generate review text based on sentiment
      if sentiment == "positive":
          review_text = f"I had a wonderful stay! The {property_type} was {random.choice(['clean', 'comfortable', 'well-equipped'])} and the location was {random.choice(['convenient', 'peaceful', 'close to amenities'])}"
      else:
          review_text = f"Unfortunately, my stay was not as expected. The {property_type} was {random.choice(['dirty', 'uncomfortable', 'poorly equipped'])} and I encountered some issues with the {random.choice(['cleanliness', 'facilities', 'communication'])}"

      # Generate response text based on sentiment + host replies to guest
      cursor.execute("SELECT guest_id FROM Bookings WHERE booking_id = ?", (booking_id,))
      guest_id = cursor.fetchone()[0]
      cursor.execute("SELECT host_id FROM Listings WHERE listing_id = ?", (listing_id,))
      host_id = cursor.fetchone()[0]

      response_chance = random.random()
      if response_chance < 0.7 and guest_id != host_id:
        if sentiment == "positive":
            response_text = "Thank you for your kind words! We're so glad you enjoyed your stay."
        elif sentiment == "negative":
            response_text = "We apologize for the inconvenience you experienced. We'll take your feedback into consideration to improve our services."
        else:
            response_text = None

        response_date = fake.date_between(start_date=review_date + timedelta(days=1), end_date=review_date + timedelta(days=14))
      else:
        response_text = None
        response_date = None

      reviews_data.append((review_id_counter, rating_score, review_text, review_date.isoformat(), response_text, response_date.isoformat() if response_date else None, booking_id, listing_id))
      review_id_counter += 1

# Insert reviews data
try:
  cursor.executemany("""
    INSERT INTO Reviews (review_id,rating_score, review_text, review_date, response_text, response_date, booking_id, listing_id)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""", reviews_data)
  conn.commit()
except sqlite3.IntegrityError:
  print(f"Error inserting user: {reviews_data}. review_id might already exist.")
except Exception as e:
  print(f"An unexpected error occurred: {e}")
conn.close()

print("Reviews table populated with fake data.")

Reviews table populated with fake data.


In [ ]:
import sqlite3
import random

# Print all data from the table
print("Reviews table data:")

conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM Reviews")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

Reviews table data:
(1, 4, 'I had a wonderful stay! The Villa was clean and the location was convenient', '2025-02-03', "Thank you for your kind words! We're so glad you enjoyed your stay.", '2025-02-15', 1, 208)
(2, 5, 'I had a wonderful stay! The House was comfortable and the location was close to amenities', '2025-02-10', None, None, 140, 240)
(3, 2, 'Unfortunately, my stay was not as expected. The House was dirty and I encountered some issues with the communication', '2024-07-28', "We apologize for the inconvenience you experienced. We'll take your feedback into consideration to improve our services.", '2024-08-06', 2, 240)
(4, 2, 'Unfortunately, my stay was not as expected. The Apartment was poorly equipped and I encountered some issues with the facilities', '2024-07-21', "We apologize for the inconvenience you experienced. We'll take your feedback into consideration to improve our services.", '2024-07-22', 313, 288)
(5, 5, 'I had a wonderful stay! The Apartment was clean and the 

### User activity table

In [ ]:
#to delete/reinitialize table
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM User_activity")
conn.commit()

In [ ]:
import sqlite3
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Connect to the database
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

# Create User_activity table (if not exists)
cursor.execute('''
CREATE TABLE IF NOT EXISTS User_activity (
  session_id INTEGER PRIMARY KEY,
  session_date TEXT,
  time_spent_page DECIMAL,
  device_type TEXT CHECK (device_type IN ('mobile_phone', 'tablet', 'desktop')),
  conversion_status TEXT CHECK (conversion_status IN ('repeating user', 'new user')),
  clicks_per_sesh INTEGER,
  user_id INTEGER NOT NULL,
  booking_success INTEGER,
  listing_clicked INTEGER,
  booking_id INTEGER,
  FOREIGN KEY (user_id) REFERENCES Users (user_id),
  FOREIGN KEY (booking_id) REFERENCES Bookings (booking_id));
''')
#time in minutes

# Get available user, listing, and booking IDs
cursor.execute("SELECT user_id FROM Users")
available_user_ids = [row[0] for row in cursor.fetchall()]

# Get confirmed booking IDs
cursor.execute("SELECT booking_id FROM Bookings WHERE booking_status = 'confirmed'")
confirmed_booking_ids = [row[0] for row in cursor.fetchall()]

# Generate user activity data
user_activity_data = []
session_id_counter = 1
for _ in range(500):
  user_id = random.choice(available_user_ids)
  session_date = fake.date_this_year().isoformat()
  time_spent_page = round(random.uniform(0.5, 60.0), 2)
  device_type = random.choice(['mobile_phone', 'tablet', 'desktop'])
  conversion_status = random.choice(['repeating user', 'new user'])
  clicks_per_sesh = random.randint(1, 20)
  listing_clicked = random.randint(0, 9)
  booking_success = random.randint(0, 1)

  if booking_success == 1:
      # First, get the confirmed booking ID, regardless of payment status
      cursor.execute("SELECT Bookings.booking_id FROM Bookings WHERE guest_id = ? AND Bookings.booking_status = 'confirmed'", (user_id,))
      booking_data = cursor.fetchall()

      if booking_data:
          booking_id = random.choice(booking_data)[0]  # Get the booking_id
      else:
          booking_id = random.choice(confirmed_booking_ids) if confirmed_booking_ids else None

  else:
      booking_id = None  # Reset booking_id if booking_success is 0

  user_activity_data.append((session_id_counter, session_date, time_spent_page, device_type,
                            conversion_status, clicks_per_sesh, user_id, booking_success, listing_clicked,
                            booking_id))
  session_id_counter += 1

try:
  cursor.executemany("""
    INSERT INTO User_activity (session_id, session_date, time_spent_page, device_type, conversion_status, clicks_per_sesh, user_id, booking_success, listing_clicked, booking_id)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", user_activity_data)
  conn.commit()
except sqlite3.IntegrityError:
  print(f"Error inserting user: {user_activity_data}. user_activity_id might already exist.")
except Exception as e:
  print(f"An unexpected error occurred: {e}")
conn.close()

print("User_activity table populated with fake data.")

User_activity table populated with fake data.


In [ ]:
import sqlite3
import random

# Print all data from the  table
print("User activity table data:")

conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM User_activity")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

User activity table data:
(1, '2025-01-03', 18.52, 'tablet', 'repeating user', 3, 130, 1, 9, 320)
(2, '2025-01-05', 37.57, 'mobile_phone', 'repeating user', 11, 180, 1, 0, 84)
(3, '2025-02-06', 27.9, 'mobile_phone', 'new user', 7, 63, 1, 8, 379)
(4, '2025-01-07', 46.31, 'tablet', 'repeating user', 20, 142, 0, 3, None)
(5, '2025-01-10', 31.96, 'desktop', 'repeating user', 14, 155, 1, 5, 119)
(6, '2025-02-24', 53.99, 'tablet', 'repeating user', 8, 388, 1, 8, 131)
(7, '2025-02-24', 20.87, 'desktop', 'new user', 12, 40, 0, 9, None)
(8, '2025-03-07', 36.77, 'desktop', 'repeating user', 1, 294, 0, 0, None)
(9, '2025-03-07', 37.87, 'tablet', 'repeating user', 18, 196, 1, 6, 17)
(10, '2025-01-01', 46.01, 'desktop', 'repeating user', 1, 146, 1, 1, 365)
(11, '2025-01-20', 55, 'tablet', 'repeating user', 15, 6, 1, 3, 254)
(12, '2025-01-17', 6.49, 'tablet', 'repeating user', 11, 252, 1, 4, 249)
(13, '2025-02-17', 43.09, 'desktop', 'new user', 14, 33, 1, 2, 10)
(14, '2025-01-09', 33.96, 'desktop', 

### Amenities table

In [ ]:
#to delete/reinitialize table
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM Amenities")
conn.commit()

In [ ]:
import sqlite3
import random

# Connect to the database
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

# Create Amenities table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Amenities (
  amenities_id INTEGER PRIMARY KEY,
  wifi TEXT CHECK (wifi IN ('yes_wifi', 'no_wifi')),
  pet_friendly TEXT CHECK (pet_friendly IN ('yes_pet_friendly', 'no_pet_friendly')),
  parking TEXT CHECK (parking IN ('yes_parking', 'no_parking')),
  fireplace TEXT CHECK (fireplace IN ('yes_fireplace', 'no_fireplace')),
  swimming_pool TEXT CHECK (swimming_pool IN ('yes_swimming_pool', 'no_swimming_pool')),
  tv TEXT CHECK (tv IN ('yes_tv', 'no_tv')),
  kettle TEXT CHECK (kettle IN ('yes_kettle', 'no_kettle')),
  fridge TEXT CHECK (fridge IN ('yes_fridge', 'no_fridge')),
  induction TEXT CHECK (induction IN ('yes_induction', 'no_induction')),
  microwave TEXT CHECK (microwave IN ('yes_microwave', 'no_microwave')),
  towels TEXT CHECK (towels IN ('yes_towels', 'no_towels')),
  laundry TEXT CHECK (laundry IN ('yes_laundry', 'no_laundry')),
  secu_camera TEXT CHECK (secu_camera IN ('yes_secu_camera', 'no_secu_camera')),
  balcony TEXT CHECK (balcony IN ('yes_balcony', 'no_balcony'))
);
''')
#0:not available 1:available

# Generate amenities data
amenities_data = []
for amenities_id in range(1,501):  # Adjust the range to control the number of amenities entries
    amenities_data.append((
        amenities_id,
        random.choice(['yes_wifi', 'no_wifi']),
        random.choice(['yes_pet_friendly', 'no_pet_friendly']),
        random.choice(['yes_parking', 'no_parking']),
        random.choice(['yes_fireplace', 'no_fireplace']),
        random.choice(['yes_swimming_pool', 'no_swimming_pool']),
        random.choice(['yes_tv', 'no_tv']),
        random.choice(['yes_kettle', 'no_kettle']),
        random.choice(['yes_fridge', 'no_fridge']),
        random.choice(['yes_induction', 'no_induction']),
        random.choice(['yes_microwave', 'no_microwave']),
        random.choice(['yes_towels', 'no_towels']),
        random.choice(['yes_laundry', 'no_laundry']),
        random.choice(['yes_secu_camera', 'no_secu_camera']),
        random.choice(['yes_balcony', 'no_balcony'])
    ))

# Insert amenities data
try:
    cursor.executemany("""
        INSERT INTO Amenities (amenities_id, wifi, pet_friendly, parking, fireplace, swimming_pool, tv, kettle, fridge, induction, microwave, towels, laundry, secu_camera, balcony)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, amenities_data)
    conn.commit()
except sqlite3.IntegrityError:
  print(f"Error inserting user: {amenities_data}. amenities_id might already exist.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Close the connection
conn.close()

print("Amenities table populated with fake data.")

Amenities table populated with fake data.


In [ ]:
import sqlite3
import random

# Print all data from the table
print("Amenities table data:")

conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM Amenities")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

Amenities table data:
(1, 'yes_wifi', 'no_pet_friendly', 'no_parking', 'no_fireplace', 'no_swimming_pool', 'yes_tv', 'no_kettle', 'no_fridge', 'yes_induction', 'no_microwave', 'yes_towels', 'no_laundry', 'yes_secu_camera', 'no_balcony')
(2, 'yes_wifi', 'yes_pet_friendly', 'no_parking', 'yes_fireplace', 'yes_swimming_pool', 'yes_tv', 'yes_kettle', 'yes_fridge', 'yes_induction', 'no_microwave', 'no_towels', 'yes_laundry', 'yes_secu_camera', 'no_balcony')
(3, 'no_wifi', 'no_pet_friendly', 'no_parking', 'no_fireplace', 'yes_swimming_pool', 'yes_tv', 'yes_kettle', 'no_fridge', 'no_induction', 'no_microwave', 'no_towels', 'no_laundry', 'yes_secu_camera', 'yes_balcony')
(4, 'no_wifi', 'yes_pet_friendly', 'no_parking', 'no_fireplace', 'no_swimming_pool', 'no_tv', 'no_kettle', 'no_fridge', 'yes_induction', 'no_microwave', 'yes_towels', 'no_laundry', 'yes_secu_camera', 'yes_balcony')
(5, 'yes_wifi', 'yes_pet_friendly', 'no_parking', 'no_fireplace', 'yes_swimming_pool', 'yes_tv', 'yes_kettle', 'y

### Refunds table

In [ ]:
#to delete/reinitialize table
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM Refunds")
conn.commit()

In [ ]:
import sqlite3
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Connect to the database
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

# Create Refunds table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Refunds (
  refund_id INTEGER PRIMARY KEY,
  refund_type TEXT CHECK (refund_type IN ('full', 'partial', 'none')),
  refund_status TEXT CHECK (refund_status IN ('pending', 'approved', 'rejected')),
  refund_amount DECIMAL,
  request_date TEXT,
  processed_date TEXT,
  booking_id INTEGER,
  payment_id INTEGER,
  FOREIGN KEY (booking_id) REFERENCES Bookings (booking_id),
  FOREIGN KEY (payment_id) REFERENCES Payments (payment_id)
);
''')

# Get available booking IDs
cursor.execute("SELECT Bookings.booking_id, Bookings.booking_date, Payments.payment_id, Payments.payment_amount FROM Bookings INNER JOIN Payments ON Bookings.booking_id = Payments.booking_id")
available_booking_payment_ids = cursor.fetchall()

# Generate refunds data
refunds_data = []
refund_id_counter = 1

for booking_id, booking_date, payment_id, payment_amount in available_booking_payment_ids:
  if random.random() < 0.3:
    refund_type = random.choice(['full', 'partial', 'none'])
   # refund_status = 'pending'  # Default to pending
    refund_amount = 0  # Default to 0
    processed_date_str = None  # Default to None

    booking_date = datetime.strptime(booking_date_str, "%Y-%m-%d").date()
    request_date = fake.date_between(start_date=booking_date + timedelta(days=1), end_date = booking_date + timedelta(days=14))
    request_date_str = request_date.strftime("%Y-%m-%d") if isinstance(request_date, datetime) else request_date

    if refund_type == 'none':
      refund_status = 'rejected'  # Directly set to rejected
      processed_date = fake.date_between(start_date=request_date, end_date=request_date + timedelta(days=14))
      processed_date_str = processed_date.strftime("%Y-%m-%d") if isinstance(processed_date, datetime) else processed_date
    elif refund_type == 'full' and booking_status =='cancelled by host':
      refund_status = 'approved'
      refund_amount = payment_amount
      processed_date = fake.date_between(start_date=request_date, end_date=request_date + timedelta(days=14))
      processed_date_str = processed_date.strftime("%Y-%m-%d") if isinstance(processed_date, datetime) else processed_date
    elif refund_type == 'partial': # and booking_status in ('cancelled by guest')
      refund_status = 'approved'
      refund_amount = payment_amount * random.uniform(0.1, 0.9)  # Some percentage of payment
      processed_date = fake.date_between(start_date=request_date, end_date=request_date + timedelta(days=14))
      processed_date_str = processed_date.strftime("%Y-%m-%d") if isinstance(processed_date, datetime) else processed_date
    else:
    # For all other cases, it's either rejected or pending with a partial refund.
      refund_status = random.choice(['pending', 'rejected'])
      if refund_status == 'rejected':
        processed_date = fake.date_between(start_date=request_date, end_date=request_date + timedelta(days=14))
        processed_date_str = processed_date.strftime("%Y-%m-%d") if isinstance(processed_date, datetime) else processed_date

    refunds_data.append((refund_id_counter, refund_type, refund_status, refund_amount, request_date_str, processed_date_str, booking_id, payment_id))
    refund_id_counter += 1


# Insert refunds data
try:
    cursor.executemany("""
        INSERT INTO Refunds (refund_id, refund_type, refund_status, refund_amount, request_date, processed_date, booking_id, payment_id)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, refunds_data)
    conn.commit()
except sqlite3.IntegrityError:
  print(f"Error inserting user: {refunds_data}. refund_id might already exist.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Close the connection
conn.close()

print("Refunds table populated with fake data.")

Refunds table populated with fake data.


In [ ]:
import sqlite3
import random

# Print all data from the table
print("Refunds table data:")

conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM Refunds")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

Refunds table data:
(1, 'full', 'rejected', 0, '2024-05-31', '2024-06-03', 2, 3)
(2, 'partial', 'approved', 11.25347055012619, '2024-05-30', '2024-06-03', 4, 5)
(3, 'full', 'pending', 0, '2024-06-02', None, 5, 6)
(4, 'none', 'rejected', 0, '2024-05-27', '2024-06-05', 9, 10)
(5, 'none', 'rejected', 0, '2024-05-23', '2024-05-29', 13, 14)
(6, 'partial', 'approved', 221.41122901209343, '2024-05-26', '2024-05-26', 22, 23)
(7, 'full', 'rejected', 0, '2024-05-30', '2024-06-06', 27, 28)
(8, 'none', 'rejected', 0, '2024-06-04', '2024-06-09', 30, 31)
(9, 'partial', 'approved', 176.30704107281142, '2024-05-28', '2024-06-09', 31, 32)
(10, 'full', 'pending', 0, '2024-05-31', None, 33, 34)
(11, 'partial', 'approved', 1147.0831716495452, '2024-05-25', '2024-06-04', 34, 35)
(12, 'none', 'rejected', 0, '2024-05-31', '2024-06-06', 37, 38)
(13, 'none', 'rejected', 0, '2024-05-29', '2024-06-11', 40, 41)
(14, 'full', 'pending', 0, '2024-05-28', None, 46, 47)
(15, 'partial', 'approved', 451.94956859936224, 

### Listing Amenities table

In [ ]:
#to delete/reinitialize table
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()
cursor.execute("DELETE FROM Listing_amenities")
conn.commit()

In [ ]:
#THIS WOULD BE TO GET 1 SET OF AMENITIES PER LISTING
import sqlite3
import random

# Connect to the database
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

# Create Listing_amenities table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Listing_amenities (
  listings_id INTEGER NOT NULL PRIMARY KEY,
  amenities_id INTEGER NOT NULL,
  FOREIGN KEY (listings_id) REFERENCES Listings (listing_id),
  FOREIGN KEY (amenities_id) REFERENCES Amenities (amenities_id)
);
''')

# Get available listings_id and amenities_id values
cursor.execute("SELECT listing_id FROM Listings")
available_listings_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT amenities_id FROM Amenities")
available_amenities_ids = [row[0] for row in cursor.fetchall()]

# Generate Listing_amenities data (modified)
listing_amenities_data = []
for listing_id in available_listings_ids:
    # Randomly select one amenities_id for this listing
    amenities_id = random.choice(available_amenities_ids)
    listing_amenities_data.append((listing_id, amenities_id))

# Insert Listing_amenities data
try:
    cursor.executemany("""
        INSERT INTO Listing_amenities (listings_id, amenities_id)
        VALUES (?, ?)
    """, listing_amenities_data)
    conn.commit()
except sqlite3.IntegrityError:
    print(f"Error inserting: {listing_amenities_data}. listings_id might already exist.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Close the connection
conn.close()

print("Listing_amenities table populated with fake data.")

Listing_amenities table populated with fake data.


In [ ]:
import sqlite3
import random

# Print all data from the table
print("Listing Amenities table data:")

conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM Listing_amenities")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

Listing Amenities table data:
(1, 326)
(2, 393)
(3, 242)
(4, 398)
(5, 243)
(6, 368)
(7, 239)
(8, 43)
(9, 192)
(10, 142)
(11, 485)
(12, 371)
(13, 456)
(14, 432)
(15, 49)
(16, 331)
(17, 470)
(18, 329)
(19, 449)
(20, 242)
(21, 359)
(22, 110)
(23, 243)
(24, 34)
(25, 349)
(26, 234)
(27, 119)
(28, 35)
(29, 141)
(30, 101)
(31, 349)
(32, 3)
(33, 392)
(34, 186)
(35, 240)
(36, 445)
(37, 58)
(38, 94)
(39, 367)
(40, 103)
(41, 252)
(42, 367)
(43, 48)
(44, 405)
(45, 91)
(46, 178)
(47, 101)
(48, 95)
(49, 179)
(50, 339)
(51, 319)
(52, 207)
(53, 212)
(54, 80)
(55, 7)
(56, 208)
(57, 72)
(58, 266)
(59, 32)
(60, 223)
(61, 15)
(62, 79)
(63, 185)
(64, 481)
(65, 439)
(66, 318)
(67, 57)
(68, 299)
(69, 258)
(70, 260)
(71, 235)
(72, 313)
(73, 164)
(74, 360)
(75, 302)
(76, 139)
(77, 495)
(78, 477)
(79, 374)
(80, 161)
(81, 160)
(82, 145)
(83, 153)
(84, 106)
(85, 349)
(86, 305)
(87, 244)
(88, 139)
(89, 480)
(90, 18)
(91, 428)
(92, 104)
(93, 170)
(94, 382)
(95, 479)
(96, 206)
(97, 168)
(98, 253)
(99, 404)
(100, 169

### Validation and accuracy checks

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("mydm.db")
cursor = conn.cursor()

#data integrity checks
def check_foreign_keys():
  #Check if foreign key references are valid
    cursor.execute("SELECT COUNT(*) FROM Users WHERE cities_id NOT IN (SELECT cities_id FROM UK_cities)")
    invalid_users = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM Listings WHERE host_id NOT IN (SELECT user_id FROM Users)")
    invalid_listings = cursor.fetchone()[0]

    print(f"Foreign Key Issues: {invalid_users} invalid Users, {invalid_listings} invalid Listings")

def check_unique_constraints():
  #Ensure unique constraints hold, such as unique emails
    cursor.execute("SELECT email, COUNT(*) FROM Users GROUP BY email HAVING COUNT(*) > 1")
    duplicates = cursor.fetchall()
    print(f"Duplicate Emails: {len(duplicates)}")

#logical data validation
def check_logical_constraints():
  #Check logical constraints like price, minimum nights, and valid dates
    cursor.execute("SELECT COUNT(*) FROM Listings WHERE price_per_night <= 0")
    invalid_prices = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM Listings WHERE min_nights < 1")
    invalid_nights = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM Listings WHERE listing_date > DATE('now')")
    future_dates = cursor.fetchone()[0]

    print(f"Logical Errors: {invalid_prices} invalid Prices, {invalid_nights} invalid Min Nights, {future_dates} future Dates")

def check_booking_constraints():
    # Check if check-out date is after check-in date
    cursor.execute("SELECT COUNT(*) FROM Bookings WHERE check_out_date <= check_in_date")
    invalid_booking_dates = cursor.fetchone()[0]

    # Check if booking date is before check-in date
    cursor.execute("SELECT COUNT(*) FROM Bookings WHERE booking_date > check_in_date")
    invalid_booking_date_order = cursor.fetchone()[0]

    print(f"Booking Constraints: {invalid_booking_dates} invalid booking date ranges, {invalid_booking_date_order} invalid booking date order")

def check_payment_constraints():
    # Check if payment amount is positive
    cursor.execute("SELECT COUNT(*) FROM Payments WHERE payment_amount <= 0")
    invalid_payment_amounts = cursor.fetchone()[0]

    print(f"Payment Constraints: {invalid_payment_amounts} invalid payment amounts")

def check_review_constraints():
    # Check if rating score is within the valid range (1-5)
    cursor.execute("SELECT COUNT(*) FROM Reviews WHERE rating_score < 1 OR rating_score > 5")
    invalid_rating_scores = cursor.fetchone()[0]

    print(f"Reviews Constraints: {invalid_rating_scores} invalid rating scores")

#cross-table consistency checks
def check_inconsistent_payment_consistency():
       # Check if total payment amount matches the sum of individual booking payments
       cursor.execute("""
           SELECT COUNT(*)
           FROM Bookings B
           JOIN Payments P ON B.booking_id = P.booking_id
           JOIN Listings L ON B.listing_id = L.listing_id  -- Join with Listings to get price_per_night
           WHERE P.payment_amount != (L.price_per_night * B.num_guests)
       """)
       inconsistent_payments = cursor.fetchone()[0]
       print(f"Cross-Table Issues: {inconsistent_payments} inconsistent payments")

def check_listing_availability():
  #check that bookings are not made on listings that aren't available
        cursor.execute("""
            SELECT COUNT(*)
            FROM Bookings B
            JOIN Listings L ON B.listing_id = L.listing_id
            WHERE L.availability_status = 'booked'
            AND B.check_in_date < L.listing_date -- Assuming listing_date is the date when it becomes available
            AND B.check_out_date > L.listing_date
        """)
        inconsistent_bookings = cursor.fetchone()[0]
        print(f"Cross-Table Issues: {inconsistent_bookings} bookings for unavailable listings.")

def check_refund_payment_consistency():
  #ensure that the refund amount does not exceed the payment amount
        cursor.execute("""
            SELECT COUNT(*)
            FROM Refunds R
            JOIN Payments P ON R.payment_id = P.payment_id
            WHERE R.refund_amount > P.payment_amount  -- Check for over-refunding
        """)
        over_refunds = cursor.fetchone()[0]
        print(f"Cross-Table Issues: {over_refunds} refunds exceeding payment amounts.")

#business rule compliance
def check_booking_rules():
       # Check if bookings for the same listing overlap (double-booking)
       cursor.execute("""
           SELECT COUNT(*)
           FROM Bookings B1, Bookings B2
           WHERE B1.listing_id = B2.listing_id
           AND B1.booking_id != B2.booking_id
           AND B1.check_in_date < B2.check_out_date
           AND B1.check_out_date > B2.check_in_date
       """)
       double_bookings = cursor.fetchone()[0]
       print(f"Bookings Rule Violations: {double_bookings} double bookings detected")

def check_listing_user_rules():
    # Listings posted by 'Guest' user type
    cursor.execute("""
        SELECT COUNT(*)
        FROM Listings L
        JOIN Users U ON L.host_id = U.user_id
        WHERE U.user_type = 'Guest'
    """)
    guest_listings = cursor.fetchone()[0]
    print(f"Listing User Rule Violations: {guest_listings} listings posted by 'Guest' users.")

def check_refund_payment_rules():
  #check that each refund is associated with a payment entry
      cursor.execute("""
        SELECT COUNT(*)
        FROM Payments P
        WHERE NOT EXISTS (SELECT 1 FROM Refunds R WHERE P.payment_id = R.payment_id) AND (booking_id, payment_amount) in (SELECT booking_id, payment_amount FROM Bookings WHERE payment_amount >0 AND check_in_date < date() AND booking_status in ('cancelled by host', 'cancelled by guest'))
    """)
      refund_missing_payments = cursor.fetchone()[0]
      print(f"Refund Payment Rule Violations: {refund_missing_payments} refund payment entries are missing.")

def run_validity_checks():
    print("Running validity checks...")
    check_foreign_keys()
    check_unique_constraints()
    check_logical_constraints()
    check_booking_constraints()
    check_payment_constraints()
    check_review_constraints()
    check_inconsistent_payment_consistency()
    check_listing_availability()
    check_refund_payment_consistency()
    check_booking_rules()
    check_listing_user_rules()
    check_refund_payment_rules()
    print("Validity checks completed.")

run_validity_checks()

# Close the connection
conn.close()

Running validity checks...
Foreign Key Issues: 0 invalid Users, 0 invalid Listings
Duplicate Emails: 0
Logical Errors: 0 invalid Prices, 0 invalid Min Nights, 0 future Dates
Booking Constraints: 0 invalid booking date ranges, 0 invalid booking date order
Payment Constraints: 0 invalid payment amounts
Reviews Constraints: 0 invalid rating scores
Cross-Table Issues: 0 inconsistent payments
Cross-Table Issues: 3 bookings for unavailable listings.
Cross-Table Issues: 0 refunds exceeding payment amounts.
Bookings Rule Violations: 16 double bookings detected
Listing User Rule Violations: 0 listings posted by 'Guest' users.
Refund Payment Rule Violations: 113 refund payment entries are missing.
Validity checks completed.


### Export to csv

In [ ]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('mydm.db')

# List of tables to export
tables_to_export = ["Users", "Listings", "Bookings", "Payments", "Reviews", "User_activity", "Amenities", "UK_cities" , "Listing_amenities", "Refunds"]

# Export each table to a separate CSV file
for table_name in tables_to_export:
    query = f"SELECT * FROM {table_name};"  # Build SQL query
    df = pd.read_sql_query(query, conn)  # Read data into Pandas DataFrame
    df.to_csv(f"{table_name}.csv", index=False)  # Export DataFrame to CSV

# Close the database connection
conn.close()

print("Specified tables exported to CSV files.")

Specified tables exported to CSV files.
